In [4]:
#TO MEET 5 PAGE LIMIT, ONLY ESSENTIAL FUNCTIONS OF INTEREST ARE PRESENT
import numpy as np; import cv2; from typing import Tuple; import scipy.signal 
##################### PREPROCESSING ####################################
def unsharp_masking(image: np.ndarray, kernel_size: Tuple[int, int] = (3,3), sigma: float = 2, weight_original: int = 5, weight_edges: int = 3, gamma: float = 2) -> np.ndarray:  
    gaussian_blurred = cv2.GaussianBlur(image, kernel_size, sigma)  # Apply Gaussian blur
    edges = cv2.subtract(image, gaussian_blurred)  # Subtract blurred image from original to find edges
    enhanced_image = cv2.addWeighted(image, weight_original, edges, weight_edges, gamma)  # Blend original image and edges
    return enhanced_image  # Return enhanced image
def apply_top_hat_transform(image: np.ndarray, kernel_size: int = 112) -> np.ndarray:  
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))  # Create elliptical structuring element
    enhanced_image = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)  # Apply top hat transformation
    return enhanced_image  # Return transformed image
def apply_high_pass_filter(image: np.ndarray) -> np.ndarray:  
    image = unsharp_masking(image)  # Apply Unsharp Masking
    blurred_image = cv2.GaussianBlur(image, (49, 49), 8)  # Blur image for a low pass effect
    high_pass_filtered_image = cv2.subtract(image, blurred_image)  # Subtract low pass filtered image to get high pass result
    return high_pass_filtered_image  # Return high pass filtered image
##################### EDGE DETECTORS ####################################
roberts_x = np.array([[1, 0],[0, -1]])
roberts_y = np.array([[0, 1], [-1, 0]])
sobel_x = np.array([[1, 0, -1],[2, 0, -2],[1, 0, -1]])
sobel_y = sobel_x.T 
def gaussian_kernel(size: int = 3, sigma: float = 1.0) -> np.ndarray: 
    size = int(size) // 2  # Ensuring the kernel size is effectively odd
    x, y = np.mgrid[-size:size+1, -size:size+1]  # Create a grid of (x, y) coordinates
    normal = 1 / (2.0 * np.pi * sigma**2)  # Normalization constant for the Gaussian function
    g = np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal  # Compute the Gaussian function
    return g / np.sum(g)  # Normalize and return the kernel
def zero_crossing(image: np.ndarray, thresh: int = 175) -> np.ndarray: 
    z_c_image = np.zeros(image.shape)  # Initialize an output image of zeros with the same shape as the input image
    h,w = image.shape  # Unpack the image dimensions into height and width
    for y in range(1, h - 1):  # Iterate over rows, excluding the border
        for x in range(1, w - 1):  # Iterate over columns, excluding the border
            patch = image[y-1:y+2, x-1:x+2]  # Extract a 3x3 patch around the current pixel
            p = image[y, x]  # Current pixel value
            maxP = patch.max()  # Maximum value in the patch
            minP = patch.min()  # Minimum value in the patch
            if (p > 0):  # Check if the current pixel is positive
                zeroCross = True if minP < 0 else False  # Determine if a zero-crossing occurs
            else:  # If the current pixel is non-positive
                zeroCross = True if maxP > 0 else False  # Determine if a zero-crossing occurs
            if ((maxP - minP) > thresh) and zeroCross:  # Check if the difference exceeds threshold and if there's a zero-crossing
                z_c_image[y, x] = 1  # Mark the current pixel in the output image
    return z_c_image  # Return the output image
def magnitude_img(G_x: np.ndarray, G_y: np.ndarray) -> np.ndarray:  
    return np.sqrt(G_x**2 + G_y**2)  # Return magnitude image
def apply_sobel_filter(image: np.ndarray, gaussian_size: int = 3, sigma: float = 1.0, threshold: int = 126) -> np.ndarray: 
  image = convert_to_grayscale(image)  # Convert image to grayscale
  image = unsharp_masking(image)  # Enhance image using unsharp masking
  g = gaussian_kernel(gaussian_size, sigma)  # Generate Gaussian kernel
  smoothed_image = scipy.signal.convolve2d(image, g, mode='same')  # Smooth image with Gaussian filter
  G_x = scipy.signal.convolve2d(smoothed_image, sobel_x, mode='same')  # Apply Sobel filter horizontally
  G_y = scipy.signal.convolve2d(smoothed_image, sobel_y, mode='same')  # Apply Sobel filter vertically
  edge_image = magnitude_img(G_x, G_y) > threshold  # Calculate edge magnitude and threshold
  return edge_image.astype(np.ndarray)  # Return edge-detected image
def apply_roberts_filter(image: np.ndarray, gaussian_size: int = 3, sigma: float = 1.0, threshold: int = 23) -> np.ndarray: 
  image = convert_to_grayscale(image)  # Convert image to grayscale
  image = unsharp_masking(image)  # Enhance image using unsharp masking
  g = gaussian_kernel(gaussian_size, sigma)  # Generate Gaussian kernel
  smoothed_image = scipy.signal.convolve2d(image, g, mode='same')  # Smooth image with Gaussian filter
  G_x = scipy.signal.convolve2d(smoothed_image, roberts_x, mode='same') 
  G_y = scipy.signal.convolve2d(smoothed_image, roberts_y, mode='same')
  edge_image = magnitude_img(G_x, G_y) > threshold  # Calculate edge magnitude and threshold
  return edge_image.astype(np.ndarray)  # Return edge-detected image
def apply_first_order_gaussian(image: np.ndarray, gaussian_size: int = 3, sigma: float = 1.5, threshold: int = 5) -> np.ndarray: 
    x, y = np.meshgrid(np.arange(-gaussian_size//2 + 1, gaussian_size//2 + 1),
                       np.arange(-gaussian_size//2 + 1, gaussian_size//2 + 1))
    g = gaussian_kernel(gaussian_size, sigma)  # Compute Gaussian kernel
    G_x = - (x / sigma**2) * g  # Calculate horizontal gradient of Gaussian
    G_y = - (y / sigma**2) * g  # Calculate vertical gradient of Gaussian
    if len(image.shape) > 2 and image.shape[2] == 3:  # If image is color
        image = convert_to_grayscale(image)  # Convert to grayscale
    image = unsharp_masking(image)  # Apply unsharp masking
    grad_x = scipy.signal.convolve2d(image, G_x, mode='same', boundary='symm')  # Convolve with G_x for horizontal gradients
    grad_y = scipy.signal.convolve2d(image, G_y, mode='same', boundary='symm')  # Convolve with G_y for vertical gradients
    magnitude = np.sqrt(grad_x**2 + grad_y**2)  # Compute magnitude of gradients
    edge_image = magnitude > threshold  # Apply threshold to get edge image
    return edge_image.astype(np.ndarray)  # Return edge-detected image
laplacian_kernel = np.array([[1,1,1],[1, -8, 1],[1,1,1]])
def apply_laplacian_filter(image: np.ndarray, threshold: int = 132) -> np.ndarray: 
  image = convert_to_grayscale(image)  # Convert image to grayscale
  image = apply_high_pass_filter(image)  # Apply high pass filter
  image = scipy.signal.convolve2d(image, laplacian_kernel, mode='same')  # Convolve with Laplacian kernel
  edge_image = zero_crossing(image, threshold)  # Apply zero crossing
  return edge_image  # Return edge-detected image
def apply_LoG_filter(image: np.ndarray, gaussian_size: int = 15, sigma: float = 3.0, threshold: int = 14) -> np.ndarray: 
    radius = gaussian_size // 2  # Compute radius from gaussian size
    x, y = np.mgrid[-radius:radius+1, -radius:radius+1]  # Generate meshgrid for LoG
    norm = x**2 + y**2  # Square norm of x, y
    log = ((norm - 2.0 * sigma**2) / sigma**4) * np.exp(-norm / (2.0 * sigma**2))  # Compute LoG
    log -= log.mean()  # Normalize to sum to 0
    image = convert_to_grayscale(image)  # Convert to grayscale
    log_image = scipy.signal.convolve2d(image, log, mode='same')  # Convolve image with LoG
    edge_image = zero_crossing(log_image, threshold)  # Detect edges with zero crossing
    return log_image, edge_image  # Return LoG filtered image and edge-detected image
def apply_gaussian_smoothing(image: np.ndarray) -> np.ndarray: 
    smoothed_image = cv2.GaussianBlur(image, (3, 3), 1)  # Blur image with Gaussian kernel
    return smoothed_image  # Return smoothed image
def calculate_color_gradient(image: np.ndarray) -> np.ndarray:  
    combined_magnitude = np.zeros_like(image[:,:,0]).astype(np.float64)  # Initialize combined gradient magnitude
    for i in range(3):  # Iterate through color channels
        grad_x = cv2.Sobel(image[:,:,i], cv2.CV_64F, 1, 0, ksize=3)  # Apply Sobel in x direction
        grad_y = cv2.Sobel(image[:,:,i], cv2.CV_64F, 0, 1, ksize=3)  # Apply Sobel in y direction
        magnitude = np.sqrt(grad_x**2 + grad_y**2)  # Compute gradient magnitude
        combined_magnitude += magnitude  # Accumulate magnitude across channels
    return combined_magnitude  # Return combined gradient magnitude
def combine_gradients(edges: np.ndarray, cg: np.ndarray) -> np.ndarray: 
    combined = edges + cg  # Sum edge gradients and color gradients
    return combined  # Return combined gradients
def convert_to_grayscale(image: np.ndarray) -> np.ndarray:  
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    return grayscale_image  # Return the grayscale image
def fog_cg_edge_detector(color_image: np.ndarray) -> np.ndarray:  
    smoothed_color = apply_gaussian_smoothing(color_image)  # Apply Gaussian smoothing to color image
    cg = calculate_color_gradient(smoothed_color)  # Calculate color gradient on smoothed image
    gray = convert_to_grayscale(color_image)  # Convert original color image to grayscale
    FoG_edges = apply_first_order_gaussian(gray)  # Apply first-order Gaussian edge detection on grayscale image
    edges = combine_gradients(FoG_edges, cg)  # Combine FoG edges with color gradients
    return edges  # Return combined edge detection result
scharr_x = np.array([[47, 0, -47], [162, 0, -162],[47, 0, -47]])
scharr_y = scharr_x.T 
def otsus_thresholding(image: np.ndarray) -> np.ndarray:  
    histogram_bins = np.arange(257)  # Create bins for histogram
    histogram, _ = np.histogram(image.ravel(), bins=histogram_bins, range=(0, 256))  # Calculate histogram
    normalized_histogram = histogram / histogram.sum()  # Normalize histogram
    bin_centers = (histogram_bins[:-1] + histogram_bins[1:]) / 2  # Find bin centers
    cumulative_sum = np.cumsum(normalized_histogram)  # Cumulative sum of histogram
    cumulative_mean = np.cumsum(bin_centers * normalized_histogram)  # Cumulative mean
    between_class_variance = np.zeros(256)  # Initialize between-class variance array
    total_mean = cumulative_mean[-1]  # Total mean of image
    for t in range(1, 256):  # Iterate over possible thresholds
        probability_class1 = cumulative_sum[t]
        probability_class2 = 1 - probability_class1
        mean_class1 = cumulative_mean[t] / probability_class1 if probability_class1 > 0 else 0
        mean_class2 = (total_mean - cumulative_mean[t]) / probability_class2 if probability_class2 > 0 else 0
        between_class_variance[t] = probability_class1 * probability_class2 * (mean_class1 - mean_class2) ** 2  # Calculate between-class variance
    optimal_threshold = np.argmax(between_class_variance)  # Optimal threshold
    thresh = image.copy()  # Copy image for thresholding
    thresh[image > optimal_threshold] = 255  # Apply threshold
    thresh[image <= optimal_threshold] = 0
    return thresh  # Return thresholded image
def apply_OSFoG(image: np.ndarray, threshold: float = .01, normalize: bool = True) -> np.ndarray: 
    FoG_CG = fog_cg_edge_detector(image)  # Apply fog and color gradient edge detector
    gray = convert_to_grayscale(image)  # Convert image to grayscale
    gray = apply_top_hat_transform(gray)  # Apply top hat transform for enhanced contrast
    binary = otsus_thresholding(gray)  # Apply Otsu's thresholding
    G_x = scipy.signal.convolve2d(binary, scharr_x, mode="same")  # Convolve with Scharr operator (x)
    G_y = scipy.signal.convolve2d(binary, scharr_y, mode="same")  # Convolve with Scharr operator (y)
    magnitude_image = magnitude_img(G_x, G_y)  # Calculate gradient magnitude
    magnitude_image += FoG_CG  # Combine with FoG_CG edges
    if normalize:
        min_val, max_val = np.min(magnitude_image), np.max(magnitude_image)  # Min and max for normalization
        magnitude_image = (magnitude_image - min_val) / (max_val - min_val)  # Normalize
    final_edges = magnitude_image > threshold  # Apply threshold to get final edges
    return final_edges  # Return final edge-detected image